**Make sure to have a copy of the database file, "world-db", downloaded and in this directory for the below to work!**

In [1]:
%load_ext sql
%sql sqlite:///./world-db

The database `world` has 3 tables: 
> * `City`
> * `Country`
> * `CountryLanguage`

Recall that to find the schema of a table, we can type the following command:

In [2]:
%sql SELECT sql FROM sqlite_master ;

 * sqlite:///./world-db
Done.


sql
"CREATE TABLE `City` ( `ID` int(11) NOT NULL, `Name` char(35) NOT NULL DEFAULT '', `CountryCode` char(3) NOT NULL DEFAULT '', `District` char(20) NOT NULL DEFAULT '', `Population` int(11) NOT NULL DEFAULT '0', PRIMARY KEY (`ID`), CONSTRAINT `city_ibfk_1` FOREIGN KEY (`CountryCode`) REFERENCES `Country` (`Code`))"
None
"CREATE TABLE `CountryLanguage` ( `CountryCode` char(3) NOT NULL DEFAULT '', `Language` char(30) NOT NULL DEFAULT '', `IsOfficial` char(1) NOT NULL DEFAULT 'F', `Percentage` float(4,1) NOT NULL DEFAULT '0.0', PRIMARY KEY (`CountryCode`,`Language`), CONSTRAINT `countryLanguage_ibfk_1` FOREIGN KEY (`CountryCode`) REFERENCES `Country` (`Code`))"
None
"CREATE TABLE `Country` ( `Code` char(3) NOT NULL DEFAULT '', `Name` char(52) NOT NULL DEFAULT '', `Continent` char(15) NOT NULL DEFAULT 'Asia', `Region` char(26) NOT NULL DEFAULT '', `SurfaceArea` float(10,2) NOT NULL DEFAULT '0.00', `IndepYear` smallint(6) DEFAULT NULL, `Population` int(11) NOT NULL DEFAULT '0', `LifeExpectancy` float(3,1) DEFAULT NULL, `GNP` float(10,2) DEFAULT NULL, `GNPOld` float(10,2) DEFAULT NULL, `LocalName` char(45) NOT NULL DEFAULT '', `GovernmentForm` char(45) NOT NULL DEFAULT '', `HeadOfState` char(60) DEFAULT NULL, `Capital` int(11) DEFAULT NULL, `Code2` char(2) NOT NULL DEFAULT '', PRIMARY KEY (`Code`))"
None
"CREATE VIEW OfficialCountryLanguage ASSELECT C.Name AS CountryName, L.Language AS LanguageFROM CountryLanguage L, Country CWHERE L.CountryCode = C.CodeAND L.IsOfficial = 'T'"
"CREATE TABLE Author( authorid INTEGER PRIMARY KEY, name TEXT )"
"CREATE TABLE Book( bookid INTEGER PRIMARY KEY, title TEXT, author INTEGER, FOREIGN KEY (author) REFERENCES Author(authorid) ON UPDATE SET NULL ON DELETE CASCADE)"


SINGLE-TABLE QUERIES
-------------------

Let's start with a simple query: what is the population of the USA?

In [3]:
%%sql 
SELECT * 
FROM Country 
WHERE Code = 'USA';

 * sqlite:///./world-db
Done.


Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
USA,United States,North America,North America,9363520.0,1776,278357000,77.1,8510700.0,8110900.0,United States,Federal Republic,George W. Bush,3813,US


In [4]:
%%sql 
SELECT Population 
FROM Country 
WHERE Code = 'USA';

 * sqlite:///./world-db
Done.


Population
278357000


**Exercise #1**: write a query that returns the countries that gained independence after 1989, along with the year they gained independence.

In [5]:
%%sql 
SELECT Code, IndepYear
FROM Country 
WHERE IndepYear > 1989;

 * sqlite:///./world-db
Done.


Code,IndepYear
ARM,1991
AZE,1991
BIH,1992
BLR,1991
CZE,1993
ERI,1993
EST,1991
FSM,1990
GEO,1991
HRV,1991


To return all attributes of a table, we use the `*` in the SELECT clause. For instance, the following query returns all the attributes for cities with population over 1 million in the US.

In [6]:
%%sql 
SELECT *
FROM City
WHERE Population >= 1000000 AND CountryCode = 'USA';

 * sqlite:///./world-db
Done.


ID,Name,CountryCode,District,Population
3793,New York,USA,New York,8008278
3794,Los Angeles,USA,California,3694820
3795,Chicago,USA,Illinois,2896016
3796,Houston,USA,Texas,1953631
3797,Philadelphia,USA,Pennsylvania,1517550
3798,Phoenix,USA,Arizona,1321045
3799,San Diego,USA,California,1223400
3800,Dallas,USA,Texas,1188580
3801,San Antonio,USA,Texas,1144646


Keep in mind that for constants we need to use single quotes!

We can rename an attribute in the output schema by using the SQL command AS. For example:

In [7]:
%%sql
select Name AS LargeUSACity
FROM City
WHERE Population >= 1000000 AND CountryCode = 'USA';

 * sqlite:///./world-db
Done.


LargeUSACity
New York
Los Angeles
Chicago
Houston
Philadelphia
Phoenix
San Diego
Dallas
San Antonio


It is possible to also perform arithmetic operations inside the SELECT clause. For example, to return the population in millions, we can use division.

In [8]:
%%sql
SELECT Name, Population / 1000000
FROM City
WHERE Population >= 1000000 AND CountryCode = 'USA';

 * sqlite:///./world-db
Done.


Name,Population / 1000000
New York,8
Los Angeles,3
Chicago,2
Houston,1
Philadelphia,1
Phoenix,1
San Diego,1
Dallas,1
San Antonio,1


Observe that division here behaves as integer division. If we want to get a float value, we can rewrite the above query (similar to type casting):

In [9]:
%%sql
SELECT Name, Population / 1000000.0
FROM City
WHERE Population >= 1000000 AND CountryCode = 'USA';


 * sqlite:///./world-db
Done.


Name,Population / 1000000.0
New York,8.008278
Los Angeles,3.69482
Chicago,2.896016
Houston,1.953631
Philadelphia,1.51755
Phoenix,1.321045
San Diego,1.2234
Dallas,1.18858
San Antonio,1.144646


Let's do some pattern matching! Find the countries that have a form of goverment that contains the word monarchy

In [10]:
%%sql
SELECT Name, GovernmentForm
FROM Country
WHERE GovernmentForm LIKE '%monarchy%';

 * sqlite:///./world-db
Done.


Name,GovernmentForm
Antigua and Barbuda,Constitutional Monarchy
Australia,"Constitutional Monarchy, Federation"
Belgium,"Constitutional Monarchy, Federation"
Bahrain,Monarchy (Emirate)
Bahamas,Constitutional Monarchy
Belize,Constitutional Monarchy
Barbados,Constitutional Monarchy
Brunei,Monarchy (Sultanate)
Bhutan,Monarchy
Canada,"Constitutional Monarchy, Federation"


Next, suppose that we want to find out all the different forms a government. Let's try the following:

In [11]:
%%sql
SELECT GovernmentForm
FROM Country ;

 * sqlite:///./world-db
Done.


GovernmentForm
Nonmetropolitan Territory of The Netherlands
Islamic Emirate
Republic
Dependent Territory of the UK
Republic
Parliamentary Coprincipality
Nonmetropolitan Territory of The Netherlands
Emirate Federation
Federal Republic
Republic


The above query did not return the desired result, because it keeps a separate copy for each occurrence of the same value in the table. To remove the duplicates, we simply need to add the DISTINCT keyword.

In [12]:
%%sql
SELECT DISTINCT GovernmentForm
FROM Country ;

 * sqlite:///./world-db
Done.


GovernmentForm
Nonmetropolitan Territory of The Netherlands
Islamic Emirate
Republic
Dependent Territory of the UK
Parliamentary Coprincipality
Emirate Federation
Federal Republic
US Territory
Co-administrated
Nonmetropolitan Territory of France


We can use ORDER BY to order the city population by decreasing population.

In [13]:
%%sql
SELECT Name, Population/1000000.0 AS Pop
FROM City
WHERE Population >= '1000000'AND CountryCode = 'USA'
ORDER BY Pop DESC;

 * sqlite:///./world-db
Done.


Name,Pop
New York,8.008278
Los Angeles,3.69482
Chicago,2.896016
Houston,1.953631
Philadelphia,1.51755
Phoenix,1.321045
San Diego,1.2234
Dallas,1.18858
San Antonio,1.144646


LIMIT is very useful if we want to look at only a few rows of a table.

In [14]:
%%sql
SELECT *
FROM City
ORDER BY Population DESC
LIMIT 3;

 * sqlite:///./world-db
Done.


ID,Name,CountryCode,District,Population
1024,Mumbai (Bombay),IND,Maharashtra,10500000
2331,Seoul,KOR,Seoul,9981619
206,S�o Paulo,BRA,S�o Paulo,9968485


**Exercise #2**: write a query that returns the two most populous cities in the US.

In [16]:
%%sql
SELECT *
FROM City
WHERE CountryCode = 'USA'
ORDER BY Population DESC
LIMIT 2;

 * sqlite:///./world-db
Done.


ID,Name,CountryCode,District,Population
3793,New York,USA,New York,8008278
3794,Los Angeles,USA,California,3694820


MULTI-TABLE QUERIES
---------------------

What are the names of all countries that speak Greek? To answer this question, we need to combine information from two tables by *joining* them.

In [17]:
%%sql
SELECT Country.Name
FROM Country, CountryLanguage
WHERE Country.Code = CountryLanguage.CountryCode
AND CountryLanguage.Language = 'Greek';

 * sqlite:///./world-db
Done.


Name
Albania
Australia
Cyprus
Germany
Greece


Notice how we need to specify the table that each attribute comes from. Another way to write the above query:

In [18]:
%%sql
SELECT Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'Greek';

 * sqlite:///./world-db
Done.


Name
Albania
Australia
Cyprus
Germany
Greece


In [22]:
%%sql
SELECT * from CountryLanguage limit 5

 * sqlite:///./world-db
Done.


CountryCode,Language,IsOfficial,Percentage
ABW,Dutch,T,5.3
ABW,English,F,9.5
ABW,Papiamento,F,76.7
ABW,Spanish,F,7.4
AFG,Balochi,F,0.9


**Exercise #3**: write a query that returns the countries that speak at least 50% Greek.

In [30]:
%%sql
SELECT * from country where code = 'USA'

 * sqlite:///./world-db
Done.


Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
USA,United States,North America,North America,9363520.0,1776,278357000,77.1,8510700.0,8110900.0,United States,Federal Republic,George W. Bush,3813,US


In [27]:
%%sql
SELECT * from city limit 1

 * sqlite:///./world-db
Done.


ID,Name,CountryCode,District,Population
1,Kabul,AFG,Kabol,1780000


In [23]:
%%sql
SELECT Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'Greek'
AND L.Percentage > 50 ;

 * sqlite:///./world-db
Done.


Name
Cyprus
Greece


**Exercise #4**: write a query that returns the district of the capital of USA.

In [40]:
%%sql 
select City.District from City , Country 
where City.CountryCode = Country.Code
and Country.Code='USA'
and City.Id = Country.Capital

 * sqlite:///./world-db
Done.


District
District of Columbia


Next, we want to return the countries that speak both Greek and English.

In [2]:
%%sql
SELECT C.Name
FROM Country C, CountryLanguage L1, CountryLanguage L2
WHERE C.Code = L1.CountryCode
AND C.Code = L2.CountryCode
AND L1.Language = 'Greek'
AND L2.Language = 'English';

 * sqlite:///./world-db
Done.


Name
Australia


Alternatively, we can write the above query using INTERSECT.

In [3]:
%%sql
SELECT C.Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'Greek'
INTERSECT
SELECT C.Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'English' ;

 * sqlite:///./world-db
Done.


Name
Australia


Which countries have population more than 100 million and do not speak English?

In [4]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Population >= 100000000
EXCEPT
SELECT C.Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'English' ;

 * sqlite:///./world-db
Done.


Name
Bangladesh
Brazil
China
India
Indonesia
Nigeria
Pakistan
Russian Federation
